In [3]:
import re
import pandas as pd
from itertools import product

def results_text_to_clipboard_df(text: str):
    """
    Parse lines like:
      [train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.29588704978190755
    Build a DF with columns: event, 5_1, 5_2, 5_3, 10_1, ... 50_3
    Copy DF to clipboard for easy paste into Sheets.
    """
    # event, lb size, set number, score
    rx = re.compile(
        r"\[train\[(.+?)_lb(\d+)_set(\d+)\]\]\s*macro-F1\s*=\s*([0-9]*\.?[0-9]+)",
        flags=re.IGNORECASE
    )

    rows = []
    for m in rx.finditer(text):
        event, lb, setnum, score = m.group(1), m.group(2), m.group(3), m.group(4)
        rows.append((event, int(lb), int(setnum), float(score)))

    if not rows:
        print("No matches found. Check the text format or regex.")
        return None

    # Build wide table
    df_long = pd.DataFrame(rows, columns=["event", "lb", "set", "score"])
    # Keep only the requested lb/set grid
    wanted_lbs  = [5, 10, 25, 50]
    wanted_sets = [1, 2, 3]
    df_long = df_long[df_long["lb"].isin(wanted_lbs) & df_long["set"].isin(wanted_sets)]

    # Pivot to event rows, columns like "5_1"
    df_wide = df_long.assign(col=lambda d: d["lb"].astype(str) + "_" + d["set"].astype(str)) \
                     .pivot_table(index="event", columns="col", values="score", aggfunc="first")

    # Ensure full cartesian column order exists
    all_cols = [f"{lb}_{s}" for lb, s in product(wanted_lbs, wanted_sets)]
    df_wide = df_wide.reindex(columns=all_cols)

    # Put 'event' as an actual column (since you asked for it in the output)
    df_out = df_wide.reset_index()

    # Copy to clipboard
    # df_out.to_clipboard(index=False)
    print("✅ Copied to clipboard. Preview:")
    print(df_out)
    return df_out

# ---- Example usage ----
text = """[train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.29588704978190755
[train[canada_wildfires_2016_lb5_set2]] macro-F1 = 0.37001663664663587
[train[canada_wildfires_2016_lb5_set3]] macro-F1 = 0.32721612383455334
[train[hurricane_irma_2017_lb5_set1]] macro-F1 = 0.4298598598700768
[train[hurricane_irma_2017_lb5_set2]] macro-F1 = 0.21567376645824105
[train[hurricane_irma_2017_lb5_set3]] macro-F1 = 0.4851931248713525
[train[hurricane_harvey_2017_lb5_set1]] macro-F1 = 0.22721826988284408
[train[hurricane_harvey_2017_lb5_set2]] macro-F1 = 0.4242294663517966
[train[hurricane_harvey_2017_lb5_set3]] macro-F1 = 0.22161465938435368
[train[kerala_floods_2018_lb5_set1]] macro-F1 = 0.31230856084426684
[train[kerala_floods_2018_lb5_set2]] macro-F1 = 0.12639010603365566
[train[kerala_floods_2018_lb5_set3]] macro-F1 = 0.15956271830080007
[train[cyclone_idai_2019_lb5_set1]] macro-F1 = 0.1360900999704298
[train[cyclone_idai_2019_lb5_set2]] macro-F1 = 0.14374275283829183
[train[cyclone_idai_2019_lb5_set3]] macro-F1 = 0.300133042180137
[train[hurricane_maria_2017_lb5_set1]] macro-F1 = 0.44969650388578625
[train[hurricane_maria_2017_lb5_set2]] macro-F1 = 0.43650882584037753
[train[hurricane_maria_2017_lb5_set3]] macro-F1 = 0.18320855238246617
[train[hurricane_dorian_2019_lb5_set1]] macro-F1 = 0.4211369919145134
[train[hurricane_dorian_2019_lb5_set2]] macro-F1 = 0.26833276871508865
[train[hurricane_dorian_2019_lb5_set3]] macro-F1 = 0.20592192097694959
[train[kaikoura_earthquake_2016_lb5_set1]] macro-F1 = 0.22872345600294836
[train[kaikoura_earthquake_2016_lb5_set2]] macro-F1 = 0.3659971447537968
[train[kaikoura_earthquake_2016_lb5_set3]] macro-F1 = 0.340456129562516
[train[hurricane_florence_2018_lb5_set1]] macro-F1 = 0.44934790974732813
[train[hurricane_florence_2018_lb5_set2]] macro-F1 = 0.12040179609031831
[train[hurricane_florence_2018_lb5_set3]] macro-F1 = 0.29195740795304864
[train[california_wildfires_2018_lb5_set1]] macro-F1 = 0.2645670937000101
[train[california_wildfires_2018_lb5_set2]] macro-F1 = 0.2995889839441398
[train[california_wildfires_2018_lb5_set3]] macro-F1 = 0.4025695494537028
[train[canada_wildfires_2016_lb10_set1]] macro-F1 = 0.44631135920068066
[train[canada_wildfires_2016_lb10_set2]] macro-F1 = 0.45024232602999387
[train[canada_wildfires_2016_lb10_set3]] macro-F1 = 0.43039304755600516
[train[hurricane_irma_2017_lb10_set1]] macro-F1 = 0.45620345075844465
[train[hurricane_irma_2017_lb10_set2]] macro-F1 = 0.5212703364825813
[train[hurricane_irma_2017_lb10_set3]] macro-F1 = 0.5082265142808642
[train[hurricane_harvey_2017_lb10_set1]] macro-F1 = 0.4223438704893665
[train[hurricane_harvey_2017_lb10_set2]] macro-F1 = 0.3130330576059006
[train[hurricane_harvey_2017_lb10_set3]] macro-F1 = 0.5656512950822628
[train[kerala_floods_2018_lb10_set1]] macro-F1 = 0.3560901150433746
[train[kerala_floods_2018_lb10_set2]] macro-F1 = 0.2635009924380152
[train[kerala_floods_2018_lb10_set3]] macro-F1 = 0.19162141688700127
[train[cyclone_idai_2019_lb10_set1]] macro-F1 = 0.37398219318887627
[train[cyclone_idai_2019_lb10_set2]] macro-F1 = 0.2956407232459741
[train[cyclone_idai_2019_lb10_set3]] macro-F1 = 0.20869080847858107
[train[hurricane_maria_2017_lb10_set1]] macro-F1 = 0.4780892517391573
[train[hurricane_maria_2017_lb10_set2]] macro-F1 = 0.45876203300631757
[train[hurricane_maria_2017_lb10_set3]] macro-F1 = 0.4852329214709501
[train[hurricane_dorian_2019_lb10_set1]] macro-F1 = 0.49721024795453683
[train[hurricane_dorian_2019_lb10_set2]] macro-F1 = 0.46089834790464895
[train[hurricane_dorian_2019_lb10_set3]] macro-F1 = 0.4740924146757963
[train[kaikoura_earthquake_2016_lb10_set1]] macro-F1 = 0.5427939798350461
[train[kaikoura_earthquake_2016_lb10_set2]] macro-F1 = 0.463655427807004
[train[kaikoura_earthquake_2016_lb10_set3]] macro-F1 = 0.548879447132173
[train[hurricane_florence_2018_lb10_set1]] macro-F1 = 0.47736548678896995
[train[hurricane_florence_2018_lb10_set2]] macro-F1 = 0.4940956920356014
[train[hurricane_florence_2018_lb10_set3]] macro-F1 = 0.4850601180693159
[train[california_wildfires_2018_lb10_set1]] macro-F1 = 0.4752622042811018
[train[california_wildfires_2018_lb10_set2]] macro-F1 = 0.4539606394598358
[train[california_wildfires_2018_lb10_set3]] macro-F1 = 0.4354122404541805
[train[canada_wildfires_2016_lb25_set1]] macro-F1 = 0.5243533068251165
[train[canada_wildfires_2016_lb25_set2]] macro-F1 = 0.5629727478853437
[train[canada_wildfires_2016_lb25_set3]] macro-F1 = 0.5597183136441853
[train[hurricane_irma_2017_lb25_set1]] macro-F1 = 0.5825373367423059
[train[hurricane_irma_2017_lb25_set2]] macro-F1 = 0.6057970733451429
[train[hurricane_irma_2017_lb25_set3]] macro-F1 = 0.5816299236453302
[train[hurricane_harvey_2017_lb25_set1]] macro-F1 = 0.6051536031721753
[train[hurricane_harvey_2017_lb25_set2]] macro-F1 = 0.6276328173962038
[train[hurricane_harvey_2017_lb25_set3]] macro-F1 = 0.603138712202646
[train[kerala_floods_2018_lb25_set1]] macro-F1 = 0.44515675910337343
[train[kerala_floods_2018_lb25_set2]] macro-F1 = 0.46470111418479715
[train[kerala_floods_2018_lb25_set3]] macro-F1 = 0.44916833115241406
[train[cyclone_idai_2019_lb25_set1]] macro-F1 = 0.4767797168662923
[train[cyclone_idai_2019_lb25_set2]] macro-F1 = 0.5093530841209095
[train[cyclone_idai_2019_lb25_set3]] macro-F1 = 0.4760470211046239
[train[hurricane_maria_2017_lb25_set1]] macro-F1 = 0.5782651956938959
[train[hurricane_maria_2017_lb25_set2]] macro-F1 = 0.6161303938022313
[train[hurricane_maria_2017_lb25_set3]] macro-F1 = 0.6497777550959712
[train[hurricane_dorian_2019_lb25_set1]] macro-F1 = 0.5383114183787803
[train[hurricane_dorian_2019_lb25_set2]] macro-F1 = 0.5694528864339327
[train[hurricane_dorian_2019_lb25_set3]] macro-F1 = 0.5335846607145538
[train[kaikoura_earthquake_2016_lb25_set1]] macro-F1 = 0.626877794108243
[train[kaikoura_earthquake_2016_lb25_set2]] macro-F1 = 0.6538285768859866
[train[kaikoura_earthquake_2016_lb25_set3]] macro-F1 = 0.6506298907557483
[train[hurricane_florence_2018_lb25_set1]] macro-F1 = 0.5726702075448233
[train[hurricane_florence_2018_lb25_set2]] macro-F1 = 0.5840446481474938
[train[hurricane_florence_2018_lb25_set3]] macro-F1 = 0.5601884398274035
[train[california_wildfires_2018_lb25_set1]] macro-F1 = 0.5572904749376952
[train[california_wildfires_2018_lb25_set2]] macro-F1 = 0.4869874774019746
[train[california_wildfires_2018_lb25_set3]] macro-F1 = 0.5387094128313052
[train[canada_wildfires_2016_lb50_set1]] macro-F1 = 0.5682871555727795
[train[canada_wildfires_2016_lb50_set2]] macro-F1 = 0.5751245320411758
[train[canada_wildfires_2016_lb50_set3]] macro-F1 = 0.5530504756126978
[train[hurricane_irma_2017_lb50_set1]] macro-F1 = 0.6450065626709055
[train[hurricane_irma_2017_lb50_set2]] macro-F1 = 0.6442002935269101
[train[hurricane_irma_2017_lb50_set3]] macro-F1 = 0.6167575343598958
[train[hurricane_harvey_2017_lb50_set1]] macro-F1 = 0.6443161217047433
[train[hurricane_harvey_2017_lb50_set2]] macro-F1 = 0.63618839058035
[train[hurricane_harvey_2017_lb50_set3]] macro-F1 = 0.6422698764226076
[train[kerala_floods_2018_lb50_set1]] macro-F1 = 0.4753342676125759
[train[kerala_floods_2018_lb50_set2]] macro-F1 = 0.5458598367159126
[train[kerala_floods_2018_lb50_set3]] macro-F1 = 0.5238131354361673
[train[cyclone_idai_2019_lb50_set1]] macro-F1 = 0.5655607783422117
[train[cyclone_idai_2019_lb50_set2]] macro-F1 = 0.5619343837231613
[train[cyclone_idai_2019_lb50_set3]] macro-F1 = 0.6116813171945445
[train[hurricane_maria_2017_lb50_set1]] macro-F1 = 0.6331796494426968
[train[hurricane_maria_2017_lb50_set2]] macro-F1 = 0.6482326734390578
[train[hurricane_maria_2017_lb50_set3]] macro-F1 = 0.6315746321146668
[train[hurricane_dorian_2019_lb50_set1]] macro-F1 = 0.583312174275839
[train[hurricane_dorian_2019_lb50_set2]] macro-F1 = 0.5765139908302197
[train[hurricane_dorian_2019_lb50_set3]] macro-F1 = 0.5787355718668081
[train[kaikoura_earthquake_2016_lb50_set1]] macro-F1 = 0.6843599486359389
[train[kaikoura_earthquake_2016_lb50_set2]] macro-F1 = 0.6799681555559395
[train[kaikoura_earthquake_2016_lb50_set3]] macro-F1 = 0.6595457955993528
[train[hurricane_florence_2018_lb50_set1]] macro-F1 = 0.6012881298048776"""
results_text_to_clipboard_df(text)


✅ Copied to clipboard. Preview:
col                      event       5_1       5_2       5_3      10_1  \
0    california_wildfires_2018  0.264567  0.299589  0.402570  0.475262   
1        canada_wildfires_2016  0.295887  0.370017  0.327216  0.446311   
2            cyclone_idai_2019  0.136090  0.143743  0.300133  0.373982   
3        hurricane_dorian_2019  0.421137  0.268333  0.205922  0.497210   
4      hurricane_florence_2018  0.449348  0.120402  0.291957  0.477365   
5        hurricane_harvey_2017  0.227218  0.424229  0.221615  0.422344   
6          hurricane_irma_2017  0.429860  0.215674  0.485193  0.456203   
7         hurricane_maria_2017  0.449697  0.436509  0.183209  0.478089   
8     kaikoura_earthquake_2016  0.228723  0.365997  0.340456  0.542794   
9           kerala_floods_2018  0.312309  0.126390  0.159563  0.356090   

col      10_2      10_3      25_1      25_2      25_3      50_1      50_2  \
0    0.453961  0.435412  0.557290  0.486987  0.538709       NaN       NaN   

col,event,5_1,5_2,5_3,10_1,10_2,10_3,25_1,25_2,25_3,50_1,50_2,50_3
0,california_wildfires_2018,0.264567,0.299589,0.402570,0.475262,0.453961,0.435412,0.557290,0.486987,0.538709,NaN,NaN,NaN
1,canada_wildfires_2016,0.295887,0.370017,0.327216,0.446311,0.450242,0.430393,0.524353,0.562973,0.559718,0.568287,0.575125,0.553050
2,cyclone_idai_2019,0.136090,0.143743,0.300133,0.373982,0.295641,0.208691,0.476780,0.509353,0.476047,0.565561,0.561934,0.611681
3,hurricane_dorian_2019,0.421137,0.268333,0.205922,0.497210,0.460898,0.474092,0.538311,0.569453,0.533585,0.583312,0.576514,0.578736
4,hurricane_florence_2018,0.449348,0.120402,0.291957,0.477365,0.494096,0.485060,0.572670,0.584045,0.560188,0.601288,NaN,NaN
5,hurricane_harvey_2017,0.227218,0.424229,0.221615,0.422344,0.313033,0.565651,0.605154,0.627633,0.603139,0.644316,0.636188,0.642270
6,hurricane_irma_2017,0.429860,0.215674,0.485193,0.456203,0.521270,0.508227,0.582537,0.605797,0.581630,0.645007,0.644200,0.616758
7,hurricane_maria_2017,0.449697,0.436509,0.183209,0.478089,0.458762,0.485233,0.578265,0.616130,0.649778,0.633180,0.648233,0.631575
8,kaikoura_earthquake_2016,0.228723,0.365997,0.340456,0.542794,0.463655,0.548879,0.626878,0.653829,0.650630,0.684360,0.679968,0.659546
9,kerala_floods_2018,0.312309,0.126390,0.159563,0.356090,0.263501,0.191621,0.445157,0.464701,0.449168,0.475334,0.545860,0.523813
